## 2. Etapa de preprocesado de texto

In [25]:
# Cargo el dataframe

import pandas as pd
import numpy as np

df_videogames = pd.read_csv("df_videogames.csv", sep=',')

# Decido realizar un primer preprocesado que elimine todo carácter que no sea una letra, creo para ello una función

!pip install stop-words
!pip install unicodedata2
!pip install regex
from stop_words import get_stop_words
import unicodedata
import re

def review_normalization(review):
    review = unicodedata.normalize('NFKD', review).lower().encode('ascii', errors='ignore').decode('utf-8')
    review = re.sub(' +', ' ', ' '.join([word if word.isalpha() else '' for word in review.split()])).strip()
    return review

In [26]:
# Relizaré un segundo nivel de preprocesado eliminando algunas stopwords a las que añado palabras derivadas de 'game', creo para ello una función
# Aplicaré en este segundo nivel de preprocesado un filtrado adicional de las stopwords de gensim
!pip install gensim
import gensim

sw_list = get_stop_words('en')
sw_list.append(['game', 'games', 'game.'])

def remove_stopwords(review, sw_list):
    review = ' '.join([word for word in review.split() if (word not in sw_list) or (word not in gensim.parsing.preprocessing.STOPWORDS)])
    return review

In [27]:
# Realizaré un tercer y último nivel de preprocesado en el que aplico una lematización simple

!pip install nltk
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatization(review):
    review = ' '.join(lemmatizer.lemmatize(word).lower().strip() for word in review.split())
    return review

# Compruebo, con un ejemplo sencillo, que la lematización funciona correctamente

lemmatizer.lemmatize('houses')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'house'

In [28]:
# Creo una función para realizar el preprocesado completo

def process_reviews(reviews, sw_list):
    processed_reviews = []

    for review in reviews:
            review = review_normalization(review)
            review = remove_stopwords(review, sw_list)
            review = lemmatization(review)
            processed_reviews.append(review)
    return processed_reviews

In [29]:
# Ejecuto el preprocesado completo

processed_reviews = process_reviews(df_videogames['reviewText'], sw_list)

In [30]:
# Observo cómo ha funcionado el preprocesado

print('Review original: {}'.format(df_videogames['reviewText'].values[0]))
print('Review procesada: {}'.format(processed_reviews[0]))


Review original: This game could have been so much more.  The hack and slash action is not even fun, it just get's tedious and boring.  I have to admit the graphic's are spectacular, but that does not make this a great game.  Rune contains lot's of jumping from object to object, which will get boring after they first introduce it into the game.  The game also had no depth whatsoever.  I never just want to play it for fun, because I felt there almost is no fun in this game.  In my opinion, if you are going to get a hack and slash action game GET BLADE OF DARKNESS.
Review procesada: game much hack slash action even just tedious admit make great rune contains jumping object will get boring first introduce game also depth never just want play felt almost fun going get hack slash action game get blade


In [31]:
# Cargo la reviews preprocesadas en el dataframe

df_videogames.loc[:,'processedreviews'] = processed_reviews

# Elimino posibles valores nulos derivados del preprocesado

df_videogames['processedreviews'] = df_videogames['processedreviews'].replace('', np.nan)
df_videogames = df_videogames.dropna()

# Elimino la variable 'reviewText', que ya no utilizaré de cara al análisis

df_videogames = df_videogames.drop('reviewText', axis = 1)

# Llegado a este punto considero que el dataframe ya está preprocesado y listo para el análisis, lo guardo en formato csv para posteriores usos

df_videogames.to_csv('df_videogames_processed.csv', index=False)

# Conclusiones

Como ya se observó en la primera etapa de exploración y visualización, el preprocesado, eliminando signos de puntuación y stopwords, ha dejado un corpus limpio, sin "ruido" y listo para el análisis. En este sentido se ha comprobado que es muy sencillo aplicar una doble eliminación de stopwords (añadimos la de gensim), así como una lematización, lo que supondrá una mayor limpieza y menor complejidad del corpus de cara a un análisis más eficiente.  